Topic classification with BERTopic

Install transformers 4.41.0 for compatability with spacy and BERTopic

`pip install transformers==4.41.0` 

Careful when running this, very hardware intensive

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import psutil
import gc
from tqdm import tqdm
import re

In [8]:
df_posts = pd.read_json('../data/dataset.json')
df_posts.head()

Apply minimal preprocessing

In [ ]:
url_pattern = re.compile(r'http\S+|www\S+|https\S+')  # Removes URLs

df_posts['text'] = df_posts['text'].apply(lambda x: url_pattern.sub('', x) if pd.notna(x) else x)

In [ ]:
df_posts = df_posts.sample(n=10000, random_state=42)  # Uncomment to test with sample

In [9]:
def monitor_memory():
    """Monitor memory usage"""
    process = psutil.Process()
    memory_gb = process.memory_info().rss / 1024 / 1024 / 1024
    return f"Memory Usage: {memory_gb:.2f} GB"

def create_multifeature_embeddings(df_posts, sentence_model, batch_size=64):
    """
    Create combined embeddings with memory monitoring and larger batches
    """
    print(f"\nStarting embedding generation for {len(df_posts)} documents")
    print(monitor_memory())
    
    def safe_join(items):
        if not items or (isinstance(items, list) and len(items) == 0):
            return ""
        return " ".join(str(item) for item in items)
    
    print("\nGenerating text embeddings...")
    text_embeddings = sentence_model.encode(
        df_posts['text'].fillna("").tolist(),
        batch_size=batch_size,
        show_progress_bar=True
    )
    print(monitor_memory())
    
    print("\nGenerating hashtag embeddings...")
    hashtag_embeddings = sentence_model.encode(
        [safe_join(tags) for tags in df_posts['hashtags']],
        batch_size=batch_size,
        show_progress_bar=True
    )
    print(monitor_memory())
    
    
    print(monitor_memory())
    
    print("\nCombining embeddings...")
    combined_embeddings = (
        0.95 * text_embeddings + 
        0.05 * hashtag_embeddings
    )
    
    # Clean up to free memory
    del text_embeddings, hashtag_embeddings
    gc.collect()
    
    print(monitor_memory())
    return combined_embeddings

def setup_bertopic_model(df_posts, batch_size=64):
    """
    Set up and train BERTopic model with memory optimization
    """
    print(f"\nDataset size: {len(df_posts)} documents")
    print(f"DataFrame memory usage: {df_posts.memory_usage().sum() / 1024**2:.2f} MB")
    print(monitor_memory())
    
    print("\nInitializing models...")
    sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Modified parameters for large dataset
    vectorizer_model = CountVectorizer(
        stop_words="english",
        #min_df=2,    # Term must appear in at least 3 documents
        #max_df=0.7,  # Ignore terms that appear in >50% of documents
        ngram_range=(1, 2)
    )
    
    # Optimized UMAP settings for large dataset
    umap_model = UMAP(
        n_neighbors=15,
        n_components=5,
        min_dist=0.0,
        metric='cosine',
        low_memory=True,
        random_state=42
    )
    
    # Create embeddings
    embeddings = create_multifeature_embeddings(df_posts, sentence_model, batch_size)
    
    # Initialize BERTopic with optimized settings
    topic_model = BERTopic(
        embedding_model=sentence_model,
        vectorizer_model=vectorizer_model,
        umap_model=umap_model,
        min_topic_size=30,  # Increased for larger dataset
        nr_topics='auto',
        calculate_probabilities=True,
        verbose=True
    )
    
    print("\nFitting BERTopic model...")
    topics, probs = topic_model.fit_transform(
        documents=df_posts['text'].fillna("").tolist(),
        embeddings=embeddings
    )
    
    return topic_model, topics, probs

def analyze_topics(topic_model, topics, df_posts):
    """
    Analyze topics with memory considerations
    """
    print("\nAnalyzing topics...")
    print(monitor_memory())
    
    topic_info = topic_model.get_topic_info()
    
    # More memory-efficient way to store topics
    df_posts['topic'] = topics
    
    # Get sample documents for each topic (limited to save memory)
    topic_docs = {}
    unique_topics = set(topics)
    print(f"\nFound {len(unique_topics)-1} topics (excluding -1)")
    
    for topic in tqdm(unique_topics):
        if topic != -1:
            topic_docs[topic] = df_posts[df_posts['topic'] == topic]['text'].head(3).tolist()
    
    try:
        print("\nGenerating visualizations...")
        topic_model.visualize_topics()
        topic_model.visualize_hierarchy()
    except Exception as e:
        print(f"Warning: Visualization error: {e}")
    
    return topic_info, topic_docs

def run_topic_analysis(df_posts, batch_size=64):
    """
    Run the complete pipeline with memory monitoring
    """
    print(f"Starting analysis with batch size: {batch_size}")
    print(monitor_memory())
    
    required_columns = ['text', 'hashtags']
    if not all(col in df_posts.columns for col in required_columns):
        raise ValueError(f"Missing columns. Required: {required_columns}")
    
    # Optionally sample for testing
    #df_posts = df_posts.sample(n=10000, random_state=42)  # Uncomment to test with sample
    
    topic_model, topics, probs = setup_bertopic_model(df_posts, batch_size)
    topic_info, topic_docs = analyze_topics(topic_model, topics, df_posts)
    
    # Create memory-efficient summary
    summary = {
        'num_topics': len(set(topics)) - 1,
        'topic_sizes': topic_info['Count'].tolist(),
        'top_topics': topic_info.head(10).to_dict('records')
    }
    
    return topic_model, summary, topics, probs

In [10]:
try:
    topic_model, summary, topics, probs = run_topic_analysis(df_posts, batch_size=64)
    
    print(f"\nAnalysis complete!")
    print(f"Found {summary['num_topics']} topics")
    print("\nTop 10 topics:")
    for topic in summary['top_topics']:
        print(f"Topic {topic['Topic']}: Size {topic['Count']}")
    
    topic_model.save("bertopic_model_large")
    
except Exception as e:
    print(f"Error during analysis: {e}")

Starting analysis with batch size: 64
Memory Usage: 0.31 GB

Dataset size: 10000 documents
DataFrame memory usage: 0.84 MB
Memory Usage: 0.32 GB

Initializing models...

Starting embedding generation for 10000 documents
Memory Usage: 0.41 GB

Generating text embeddings...


Batches: 100%|██████████| 157/157 [00:10<00:00, 15.02it/s]


Memory Usage: 0.85 GB

Generating hashtag embeddings...


Batches: 100%|██████████| 157/157 [00:09<00:00, 16.91it/s]
2025-01-23 12:25:00,137 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Memory Usage: 1.20 GB
Memory Usage: 1.20 GB

Combining embeddings...
Memory Usage: 1.24 GB

Fitting BERTopic model...


2025-01-23 12:25:09,952 - BERTopic - Dimensionality - Completed ✓
2025-01-23 12:25:09,952 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-23 12:25:10,591 - BERTopic - Cluster - Completed ✓
2025-01-23 12:25:10,591 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-23 12:25:10,914 - BERTopic - Representation - Completed ✓
2025-01-23 12:25:10,915 - BERTopic - Topic reduction - Reducing number of topics
2025-01-23 12:25:11,231 - BERTopic - Topic reduction - Reduced number of topics from 53 to 28



Analyzing topics...
Memory Usage: 1.35 GB

Found 27 topics (excluding -1)


100%|██████████| 28/28 [00:00<00:00, 7755.94it/s]
2025-01-23 12:25:11,457 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.



Generating visualizations...

Analysis complete!
Found 27 topics

Top 10 topics:
Topic -1: Size 4502
Topic 0: Size 1011
Topic 1: Size 914
Topic 2: Size 849
Topic 3: Size 616
Topic 4: Size 308
Topic 5: Size 277
Topic 6: Size 217
Topic 7: Size 184
Topic 8: Size 114


In [11]:
# 1. Save visualizations to HTML files
fig = topic_model.visualize_barchart(top_n_topics=20)
fig.write_html("../output/topic_barchart.html")

topic_model.visualize_topics().write_html("../output/topic_clusters.html")
topic_model.visualize_hierarchy().write_html("../output/topic_hierarchy.html")

# 2. Print text-based summary
topics_info = topic_model.get_topic_info()
print("\nMost frequent topics with their terms:")
for _, row in topics_info.head(20).iterrows():
    topic_id = row['Topic']
    size = row['Count']
    if topic_id != -1:
        terms = topic_model.get_topic(topic_id)
        print(f"\nTopic {topic_id} (Size: {size}):")
        # Print top 10 terms for each topic with their weights
        for term, weight in terms[:20]:
            print(f"  - {term}: {weight:.3f}")


Most frequent topics with their terms:

Topic 0 (Size: 1011):
  - vote: 0.042
  - artist: 0.037
  - voting: 0.030
  - social artist: 0.029
  - let: 0.027
  - social: 0.027
  - btsbbmas: 0.026
  - bbmas: 0.024
  - army: 0.023
  - amas: 0.022

Topic 1 (Size: 914):
  - politics: 0.041
  - political: 0.017
  - time: 0.012
  - trump: 0.011
  - people: 0.010
  - like: 0.009
  - truss: 0.009
  - corruption: 0.009
  - truth: 0.008
  - just: 0.008

Topic 2 (Size: 849):
  - health: 0.050
  - mental: 0.038
  - mental health: 0.036
  - healthcare: 0.023
  - mentalhealth: 0.019
  - covid: 0.016
  - mentalhealthawareness: 0.015
  - care: 0.013
  - secretary: 0.013
  - medical: 0.012

Topic 3 (Size: 616):
  - business: 0.048
  - marketing: 0.019
  - entrepreneurship: 0.018
  - check: 0.015
  - success: 0.014
  - online: 0.013
  - boost: 0.012
  - sales: 0.011
  - looking: 0.011
  - social media: 0.010

Topic 4 (Size: 308):
  - science: 0.094
  - research: 0.017
  - scientific: 0.015
  - knowledge: 0

In [12]:
# Add topic numbers and labels to dataframe
df_posts['topic_num'] = topics

# Get topic labels (using top 3 terms for each topic)
topic_labels = {}
for topic_id in set(topics):
    if topic_id != -1:  # Skip outlier topic -1
        terms = topic_model.get_topic(topic_id)
        label = ", ".join([term for term, _ in terms[:3]])
        topic_labels[topic_id] = label
    else:
        topic_labels[topic_id] = "Other"

# Add descriptive topic labels
df_posts['topic_label'] = df_posts['topic_num'].map(topic_labels)

# Save augmented dataframe
df_posts.to_csv('../output/posts_with_topics.csv', index=False)